In [1]:
# 직접 vocab을 만드는 파일
import pandas as pd
dataset = pd.read_csv("final.csv")
dataset.dropna(inplace = True)
dataset.reset_index(inplace = True)
dataset.drop(["index"], inplace = True, axis = 1)

In [4]:
# BertWordPieceTokenizer를 통해서 일단 직접 tokenize를 진행
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(
    # 원래는 vocab이 들어가야 하지만 현재는 vocab을 만들어내야 하기 때문에 따로 지정하지 않는다.
    None,
    clean_text = True,
    handle_chinese_chars = True,
    strip_accents = False,
    lowercase = False,
    wordpieces_prefix = "##"
)

In [7]:
from tqdm import tqdm
# encoder_input_train에는 tokenize하고자 하는 문장들이 들어가게 된다
# ex) ["나는 밥을 먹는다", "반찬은 고기다", "배가 불렀다"] -> 해당 리스트가 encode 대상이다.
encoder_input_train = []
for i in tqdm(range(len(dataset))):
    encoder_input_train.append(dataset.loc[i, "PAPER_TEXT"])

100%|██████████| 54242/54242 [00:00<00:00, 154568.34it/s]


In [9]:
# encoder_input_train으로 구성한 텍스트셋을 학습을 통해서 vocab_size 크기 만큼 vocab을 만들어낸다.
# 1차적으로 MyCanvas 파일의 vocab.txt는 8002개로 만들어냈다.
# vocab_size = n -> n을 통해서 직접 본인이 원하는 tokenize를 진행할 수 있다.
tokenizer.train_from_iterator(
    encoder_input_train,
    vocab_size=8002,
    min_frequency=2,
    show_progress=True,
    special_tokens = ["[PAD]", "[CLS]", "[UNK]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##",
)

In [17]:
import numpy as np
# txt 파일 디렉터리 및 파일명 지정
vocab_dir = "./vocab.txt"
vocab = tokenizer.get_vocab()
vocabulary = [[v, k] for k, v in vocab.items()]
vocabulary = sorted(vocabulary)
vocabulary = list(np.array(vocabulary)[:, 1])

# vocabulary 저장
with open(vocab_dir, 'w+') as lf:
    lf.write('\n'.join(vocabulary))
    
# 여기까지가 vocab.txt를 만들어내는 과정이다.

In [24]:
# BertTokenizer 확인하는 과정
# 총 2가지의 Custom Berttokenizer를 만들어낼 수 있다.
# 밑의 2가지 방법의 공통사항: do_basic_tokenize = False로 지정해야 한다.
# 이는 우리가 하고자 하는 task에 그대로 똑같이 적용하면 된다.
from transformers import BertTokenizer

# bert_tokenizer = BertTokenizer("./vocab.txt", do_basic_tokenize = False)
bert_tokenizer = BertTokenizer.from_pretrained("./vocab.txt", do_basic_tokenize = False)

/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1642: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
